# Notebook 10: Enterprise MCP Patterns

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 10 of 13  
**Level:** Advanced  
**Duration:** ~75 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Understand enterprise requirements for MCP deployments
2. Design multi-tenant MCP architectures
3. Implement compliance and audit patterns
4. Apply real-world patterns from Red Hat, Apigee, and banking
5. Build production-ready security controls
6. Design for scale and high availability

## 1. Enterprise Requirements Overview

### What Makes Enterprise Different?

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ENTERPRISE MCP REQUIREMENTS                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   🔒 SECURITY                      📋 COMPLIANCE                        │
│   ══════════                       ════════════                         │
│   • Zero-trust architecture        • Audit logging (SOC2, HIPAA)        │
│   • mTLS between services          • Data residency requirements        │
│   • Secret rotation                • PII handling policies              │
│   • Input sanitization             • Retention policies                 │
│   • Output filtering               • Access reviews                     │
│                                                                          │
│   👥 MULTI-TENANCY                 📈 SCALE                             │
│   ════════════════                 ═══════                              │
│   • Tenant isolation               • Horizontal scaling                 │
│   • Per-tenant configuration       • Load balancing                     │
│   • Resource quotas                • Connection pooling                 │
│   • Custom tool sets               • Caching strategies                 │
│                                                                          │
│   🔄 RELIABILITY                   📊 OBSERVABILITY                     │
│   ═════════════                    ══════════════                       │
│   • High availability (99.9%+)     • Distributed tracing                │
│   • Circuit breakers               • Metrics & alerting                 │
│   • Graceful degradation           • Log aggregation                    │
│   • Disaster recovery              • Cost attribution                   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup for this notebook
import json
from typing import Dict, List, Any, Optional, Set
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from enum import Enum
import hashlib
import uuid

def pprint(obj: dict, title: str = None):
    if title:
        print(f"\n{'='*60}")
        print(f" {title}")
        print(f"{'='*60}")
    print(json.dumps(obj, indent=2, default=str))

## 2. Multi-Tenant Architecture Patterns

### Pattern 1: Shared Gateway, Isolated Backends

```
┌─────────────────────────────────────────────────────────────────────────┐
│              PATTERN 1: SHARED GATEWAY, ISOLATED BACKENDS                │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   Tenant A ──┐                                                          │
│              │      ┌──────────────┐      ┌─────────────────────┐       │
│   Tenant B ──┼─────►│              │      │ Tenant A Backends   │       │
│              │      │   SHARED     │─────►│ • github-a          │       │
│   Tenant C ──┘      │   GATEWAY    │      │ • database-a        │       │
│                     │              │      └─────────────────────┘       │
│                     │ • Auth       │                                    │
│                     │ • Routing    │      ┌─────────────────────┐       │
│                     │ • Logging    │─────►│ Tenant B Backends   │       │
│                     │              │      │ • github-b          │       │
│                     └──────────────┘      │ • database-b        │       │
│                                           └─────────────────────┘       │
│                                                                          │
│   ✅ Pros: Cost efficient, centralized management                       │
│   ❌ Cons: Gateway is single point of failure                           │
│   📋 Use: Different departments in same org                             │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Pattern 2: Tenant-Specific Gateways

```
┌─────────────────────────────────────────────────────────────────────────┐
│              PATTERN 2: TENANT-SPECIFIC GATEWAYS                         │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   Tenant A ────►┌──────────────┐────►┌─────────────────────┐            │
│                 │ Gateway A    │     │ Tenant A Backends   │            │
│                 │ (dedicated)  │     └─────────────────────┘            │
│                 └──────────────┘                                        │
│                                                                          │
│   Tenant B ────►┌──────────────┐────►┌─────────────────────┐            │
│                 │ Gateway B    │     │ Tenant B Backends   │            │
│                 │ (dedicated)  │     └─────────────────────┘            │
│                 └──────────────┘                                        │
│                                                                          │
│   ✅ Pros: Full isolation, custom configs per tenant                    │
│   ❌ Cons: Higher cost, more operational overhead                       │
│   📋 Use: Enterprise customers with strict isolation needs              │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Pattern 3: Federated Gateways (Hybrid)

```
┌─────────────────────────────────────────────────────────────────────────┐
│              PATTERN 3: FEDERATED GATEWAYS                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│                      ┌──────────────────┐                               │
│   All Tenants ──────►│  GLOBAL GATEWAY  │                               │
│                      │  • Auth          │                               │
│                      │  • Routing       │                               │
│                      │  • Shared tools  │                               │
│                      └────────┬─────────┘                               │
│                               │                                         │
│              ┌────────────────┼────────────────┐                        │
│              │                │                │                        │
│              ▼                ▼                ▼                        │
│   ┌──────────────┐  ┌──────────────┐  ┌──────────────┐                 │
│   │ Regional GW  │  │ Regional GW  │  │ Regional GW  │                 │
│   │ (US-East)    │  │ (EU-West)    │  │ (APAC)       │                 │
│   └──────┬───────┘  └──────┬───────┘  └──────┬───────┘                 │
│          │                 │                 │                          │
│          ▼                 ▼                 ▼                          │
│   [US Backends]     [EU Backends]     [APAC Backends]                   │
│                                                                          │
│   ✅ Pros: Data residency compliance, low latency                       │
│   ❌ Cons: Complex, requires cross-region coordination                  │
│   📋 Use: Global enterprises with data sovereignty requirements         │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Multi-Tenant Configuration Model

@dataclass
class TenantConfig:
    """Configuration for a tenant in multi-tenant MCP."""
    tenant_id: str
    name: str
    tier: str = "standard"  # "free", "standard", "enterprise"
    
    # Resource limits
    max_requests_per_minute: int = 100
    max_concurrent_connections: int = 10
    max_tool_calls_per_day: int = 10000
    
    # Feature flags
    features: Set[str] = field(default_factory=set)
    
    # Tool access
    allowed_tools: Optional[Set[str]] = None  # None = all
    denied_tools: Set[str] = field(default_factory=set)
    
    # Backend routing
    dedicated_backends: Dict[str, str] = field(default_factory=dict)
    
    # Compliance
    data_region: str = "us-east-1"
    retention_days: int = 90
    pii_masking: bool = True


class TenantManager:
    """Manages multi-tenant configuration and routing."""
    
    def __init__(self):
        self.tenants: Dict[str, TenantConfig] = {}
        self.usage: Dict[str, Dict[str, int]] = {}  # tenant_id -> {metric: count}
    
    def register_tenant(self, config: TenantConfig):
        """Register a new tenant."""
        self.tenants[config.tenant_id] = config
        self.usage[config.tenant_id] = {
            "requests_today": 0,
            "tool_calls_today": 0,
            "active_connections": 0
        }
    
    def get_tenant(self, tenant_id: str) -> Optional[TenantConfig]:
        """Get tenant configuration."""
        return self.tenants.get(tenant_id)
    
    def check_quota(self, tenant_id: str, metric: str) -> bool:
        """Check if tenant is within quota."""
        tenant = self.tenants.get(tenant_id)
        if not tenant:
            return False
        
        usage = self.usage.get(tenant_id, {})
        
        if metric == "tool_calls":
            return usage.get("tool_calls_today", 0) < tenant.max_tool_calls_per_day
        elif metric == "connections":
            return usage.get("active_connections", 0) < tenant.max_concurrent_connections
        
        return True
    
    def record_usage(self, tenant_id: str, metric: str, count: int = 1):
        """Record usage for a tenant."""
        if tenant_id not in self.usage:
            self.usage[tenant_id] = {}
        
        current = self.usage[tenant_id].get(metric, 0)
        self.usage[tenant_id][metric] = current + count
    
    def get_backend_for_tenant(self, tenant_id: str, service: str) -> Optional[str]:
        """Get the backend URL for a tenant's service."""
        tenant = self.tenants.get(tenant_id)
        if not tenant:
            return None
        
        # Check for dedicated backend
        if service in tenant.dedicated_backends:
            return tenant.dedicated_backends[service]
        
        # Return shared backend based on region
        return f"https://{tenant.data_region}.mcp.internal/{service}"


# Demo
manager = TenantManager()

# Register tenants
manager.register_tenant(TenantConfig(
    tenant_id="acme-corp",
    name="ACME Corporation",
    tier="enterprise",
    max_requests_per_minute=1000,
    max_tool_calls_per_day=100000,
    features={"advanced-analytics", "custom-tools", "priority-support"},
    data_region="eu-west-1",
    pii_masking=True
))

manager.register_tenant(TenantConfig(
    tenant_id="startup-xyz",
    name="Startup XYZ",
    tier="standard",
    max_requests_per_minute=100,
    max_tool_calls_per_day=10000,
    features={"basic-tools"},
    data_region="us-east-1"
))

print("Multi-Tenant Configuration:")
print("=" * 60)

for tenant_id, tenant in manager.tenants.items():
    print(f"\n{tenant.name} ({tenant_id})")
    print(f"  Tier: {tenant.tier}")
    print(f"  Rate limit: {tenant.max_requests_per_minute}/min")
    print(f"  Daily tool calls: {tenant.max_tool_calls_per_day:,}")
    print(f"  Region: {tenant.data_region}")
    print(f"  Features: {tenant.features}")

## 3. Compliance and Audit Patterns

### Comprehensive Audit Logging

In [ ]:
# Enterprise Audit Logging

class AuditEventType(Enum):
    AUTHENTICATION = "authentication"
    AUTHORIZATION = "authorization"
    TOOL_CALL = "tool_call"
    RESOURCE_ACCESS = "resource_access"
    CONFIGURATION_CHANGE = "configuration_change"
    ERROR = "error"


@dataclass
class AuditEvent:
    """Comprehensive audit event for compliance."""
    event_id: str
    timestamp: datetime
    event_type: AuditEventType
    
    # Actor information
    tenant_id: str
    user_id: str
    user_email: Optional[str] = None
    user_roles: Set[str] = field(default_factory=set)
    
    # Request context
    request_id: str = ""
    session_id: str = ""
    client_ip: str = ""
    user_agent: str = ""
    
    # Action details
    action: str = ""
    resource: str = ""
    
    # Input/Output (sanitized)
    input_hash: str = ""  # Hash of input, not raw data
    output_summary: str = ""
    
    # Result
    success: bool = True
    error_code: Optional[str] = None
    error_message: Optional[str] = None
    
    # Performance
    duration_ms: float = 0
    
    # Compliance metadata
    data_classification: str = "internal"  # public, internal, confidential, restricted
    pii_accessed: bool = False
    gdpr_relevant: bool = False


class AuditLogger:
    """Enterprise audit logger with compliance features."""
    
    def __init__(self, retention_days: int = 90):
        self.retention_days = retention_days
        self.events: List[AuditEvent] = []
        self.pii_patterns = [
            r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',  # Email
            r'\b\d{3}-\d{2}-\d{4}\b',  # SSN
            r'\b\d{16}\b',  # Credit card
        ]
    
    def log(self, event: AuditEvent):
        """Log an audit event."""
        self.events.append(event)
        # In production: send to SIEM, write to immutable storage
    
    def log_tool_call(
        self,
        tenant_id: str,
        user_id: str,
        tool_name: str,
        arguments: Dict[str, Any],
        result: Any,
        success: bool,
        duration_ms: float,
        **kwargs
    ) -> AuditEvent:
        """Log a tool call with proper sanitization."""
        # Hash the input instead of storing raw data
        input_hash = hashlib.sha256(
            json.dumps(arguments, sort_keys=True).encode()
        ).hexdigest()
        
        # Summarize output without sensitive data
        output_summary = self._summarize_output(result)
        
        # Check for PII access
        pii_accessed = self._check_pii_access(arguments, result)
        
        event = AuditEvent(
            event_id=str(uuid.uuid4()),
            timestamp=datetime.now(),
            event_type=AuditEventType.TOOL_CALL,
            tenant_id=tenant_id,
            user_id=user_id,
            action=f"tool_call:{tool_name}",
            resource=tool_name,
            input_hash=input_hash,
            output_summary=output_summary,
            success=success,
            duration_ms=duration_ms,
            pii_accessed=pii_accessed,
            **kwargs
        )
        
        self.log(event)
        return event
    
    def _summarize_output(self, result: Any) -> str:
        """Create a safe summary of output."""
        if result is None:
            return "null"
        if isinstance(result, dict):
            return f"dict with {len(result)} keys"
        if isinstance(result, list):
            return f"list with {len(result)} items"
        if isinstance(result, str):
            return f"string of {len(result)} chars"
        return str(type(result).__name__)
    
    def _check_pii_access(self, arguments: Dict, result: Any) -> bool:
        """Check if PII was potentially accessed."""
        import re
        text = json.dumps(arguments) + json.dumps(result, default=str)
        for pattern in self.pii_patterns:
            if re.search(pattern, text):
                return True
        return False
    
    def query_events(
        self,
        tenant_id: Optional[str] = None,
        user_id: Optional[str] = None,
        event_type: Optional[AuditEventType] = None,
        start_time: Optional[datetime] = None,
        end_time: Optional[datetime] = None
    ) -> List[AuditEvent]:
        """Query audit events with filters."""
        results = self.events
        
        if tenant_id:
            results = [e for e in results if e.tenant_id == tenant_id]
        if user_id:
            results = [e for e in results if e.user_id == user_id]
        if event_type:
            results = [e for e in results if e.event_type == event_type]
        if start_time:
            results = [e for e in results if e.timestamp >= start_time]
        if end_time:
            results = [e for e in results if e.timestamp <= end_time]
        
        return results
    
    def generate_compliance_report(
        self,
        tenant_id: str,
        start_date: datetime,
        end_date: datetime
    ) -> Dict[str, Any]:
        """Generate a compliance report for auditors."""
        events = self.query_events(
            tenant_id=tenant_id,
            start_time=start_date,
            end_time=end_date
        )
        
        return {
            "report_generated": datetime.now().isoformat(),
            "tenant_id": tenant_id,
            "period": {
                "start": start_date.isoformat(),
                "end": end_date.isoformat()
            },
            "summary": {
                "total_events": len(events),
                "successful_events": len([e for e in events if e.success]),
                "failed_events": len([e for e in events if not e.success]),
                "pii_access_events": len([e for e in events if e.pii_accessed]),
                "unique_users": len(set(e.user_id for e in events)),
            },
            "events_by_type": {
                t.value: len([e for e in events if e.event_type == t])
                for t in AuditEventType
            }
        }


# Demo
audit = AuditLogger(retention_days=365)  # 1 year for compliance

# Simulate some audit events
audit.log_tool_call(
    tenant_id="acme-corp",
    user_id="alice@acme.com",
    tool_name="query_database",
    arguments={"sql": "SELECT * FROM users WHERE email = 'test@example.com'"},
    result={"rows": [{"id": 1, "name": "Test User"}]},
    success=True,
    duration_ms=45.2,
    client_ip="10.0.0.50"
)

audit.log_tool_call(
    tenant_id="acme-corp",
    user_id="bob@acme.com",
    tool_name="delete_record",
    arguments={"table": "users", "id": 123},
    result=None,
    success=False,
    duration_ms=12.1,
    error_code="UNAUTHORIZED",
    error_message="User lacks delete permission"
)

print("Audit Logging Demo:")
print("=" * 60)
print(f"\nTotal events logged: {len(audit.events)}")

# Generate report
report = audit.generate_compliance_report(
    tenant_id="acme-corp",
    start_date=datetime.now() - timedelta(days=1),
    end_date=datetime.now()
)
print("\nCompliance Report:")
print(json.dumps(report, indent=2))

## 4. DeployGuard: Enterprise Security Framework

Based on the MCP research documents, DeployGuard provides comprehensive security:

In [ ]:
# DeployGuard Security Framework
DEPLOYGUARD_ARCHITECTURE = '''
┌─────────────────────────────────────────────────────────────────────────┐
│                    DEPLOYGUARD SECURITY FRAMEWORK                        │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌────────────────────────────────────────────────────────────┐        │
│   │                     DEPLOYGUARD GATEWAY                     │        │
│   │                                                             │        │
│   │  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐        │        │
│   │  │   AUTH      │  │   POLICY    │  │   AUDIT     │        │        │
│   │  │   ENGINE    │  │   ENGINE    │  │   ENGINE    │        │        │
│   │  │             │  │             │  │             │        │        │
│   │  │ • OAuth 2.1 │  │ • RBAC      │  │ • Log all   │        │        │
│   │  │ • mTLS      │  │ • Tool ACLs │  │ • Immutable │        │        │
│   │  │ • API Keys  │  │ • Rate Limit│  │ • Compliant │        │        │
│   │  └─────────────┘  └─────────────┘  └─────────────┘        │        │
│   │                                                             │        │
│   │  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐        │        │
│   │  │   INPUT     │  │   OUTPUT    │  │   RUNTIME   │        │        │
│   │  │   FILTER    │  │   FILTER    │  │   SANDBOX   │        │        │
│   │  │             │  │             │  │             │        │        │
│   │  │ • Injection │  │ • PII mask  │  │ • Isolation │        │        │
│   │  │ • Validation│  │ • Data class│  │ • Timeouts  │        │        │
│   │  │ • Schema    │  │ • Redaction │  │ • Resources │        │        │
│   │  └─────────────┘  └─────────────┘  └─────────────┘        │        │
│   │                                                             │        │
│   └────────────────────────────────────────────────────────────┘        │
│                                │                                        │
│                                ▼                                        │
│                    ┌───────────────────────┐                            │
│                    │    MCP SERVERS        │                            │
│                    │    (Protected)        │                            │
│                    └───────────────────────┘                            │
│                                                                          │
│   KEY FEATURES:                                                         │
│   • Defense in depth - multiple security layers                         │
│   • Zero trust - verify every request                                   │
│   • Least privilege - minimal access by default                         │
│   • Audit everything - comprehensive logging                            │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
'''

print(DEPLOYGUARD_ARCHITECTURE)

In [ ]:
# DeployGuard Implementation Components

class InputFilter:
    """Filters and validates input to MCP tools."""
    
    # Common injection patterns to block
    INJECTION_PATTERNS = [
        r"(?i)(<script|javascript:|on\w+=)",  # XSS
        r"(?i)(union\s+select|;\s*drop|;\s*delete)",  # SQL injection
        r"(?i)(\$\{|\{\{|<%)",  # Template injection
        r"(?i)(\.\./|\.\.\\)",  # Path traversal
    ]
    
    def __init__(self):
        import re
        self.patterns = [re.compile(p) for p in self.INJECTION_PATTERNS]
    
    def check_injection(self, value: str) -> tuple[bool, str]:
        """Check for injection attacks."""
        import re
        for i, pattern in enumerate(self.patterns):
            if pattern.search(value):
                return False, f"Potential injection detected (pattern {i})"
        return True, "OK"
    
    def validate_input(self, arguments: Dict[str, Any]) -> tuple[bool, List[str]]:
        """Validate all input arguments."""
        errors = []
        
        def check_value(key: str, value: Any):
            if isinstance(value, str):
                safe, msg = self.check_injection(value)
                if not safe:
                    errors.append(f"{key}: {msg}")
            elif isinstance(value, dict):
                for k, v in value.items():
                    check_value(f"{key}.{k}", v)
            elif isinstance(value, list):
                for i, v in enumerate(value):
                    check_value(f"{key}[{i}]", v)
        
        for key, value in arguments.items():
            check_value(key, value)
        
        return len(errors) == 0, errors


class OutputFilter:
    """Filters sensitive data from MCP tool outputs."""
    
    PII_PATTERNS = {
        "email": r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
        "ssn": r'\b\d{3}-\d{2}-\d{4}\b',
        "credit_card": r'\b(?:\d{4}[- ]?){3}\d{4}\b',
        "phone": r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b',
    }
    
    def __init__(self, mask_char: str = "*"):
        import re
        self.mask_char = mask_char
        self.patterns = {
            name: re.compile(pattern) 
            for name, pattern in self.PII_PATTERNS.items()
        }
    
    def mask_pii(self, text: str) -> str:
        """Mask PII in text."""
        import re
        result = text
        for name, pattern in self.patterns.items():
            def replacer(match):
                value = match.group()
                if name == "email":
                    # Keep domain, mask username
                    parts = value.split("@")
                    return f"{self.mask_char * len(parts[0])}@{parts[1]}"
                else:
                    # Keep last 4 chars
                    return self.mask_char * (len(value) - 4) + value[-4:]
            
            result = pattern.sub(replacer, result)
        return result
    
    def filter_output(self, result: Any) -> Any:
        """Filter sensitive data from output."""
        if isinstance(result, str):
            return self.mask_pii(result)
        elif isinstance(result, dict):
            return {k: self.filter_output(v) for k, v in result.items()}
        elif isinstance(result, list):
            return [self.filter_output(v) for v in result]
        return result


# Demo
input_filter = InputFilter()
output_filter = OutputFilter()

print("DeployGuard Security Filters:")
print("=" * 60)

# Test input filtering
print("\n--- Input Filter Tests ---")
test_inputs = [
    {"query": "SELECT * FROM users"},  # Safe
    {"query": "SELECT * FROM users; DROP TABLE users;"},  # SQL injection
    {"path": "../../../etc/passwd"},  # Path traversal
    {"name": "<script>alert('xss')</script>"},  # XSS
]

for args in test_inputs:
    safe, errors = input_filter.validate_input(args)
    status = "✅ Safe" if safe else f"❌ Blocked: {errors}"
    print(f"  {list(args.values())[0][:40]}... → {status}")

# Test output filtering
print("\n--- Output Filter Tests ---")
test_output = {
    "user": "John Doe",
    "email": "john.doe@example.com",
    "ssn": "123-45-6789",
    "phone": "555-123-4567",
    "card": "4111-1111-1111-1111"
}

filtered = output_filter.filter_output(test_output)
print(f"  Original: {test_output}")
print(f"  Filtered: {filtered}")

## 5. Real-World Pattern: Red Hat OpenShift AI

From the research docs - how Red Hat implements MCP in OpenShift AI:

In [ ]:
# Red Hat OpenShift AI MCP Pattern
OPENSHIFT_PATTERN = '''
┌─────────────────────────────────────────────────────────────────────────┐
│              RED HAT OPENSHIFT AI - MCP INTEGRATION                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────────────────────────────────────────────────────────┐       │
│   │                    OPENSHIFT CLUSTER                         │       │
│   │                                                              │       │
│   │   ┌──────────────────┐    ┌──────────────────┐              │       │
│   │   │   AI WORKBENCH   │    │   MODEL SERVER   │              │       │
│   │   │   (Notebooks)    │    │   (vLLM/TGI)     │              │       │
│   │   └────────┬─────────┘    └────────┬─────────┘              │       │
│   │            │                       │                        │       │
│   │            └───────────┬───────────┘                        │       │
│   │                        │                                    │       │
│   │                        ▼                                    │       │
│   │            ┌──────────────────────┐                         │       │
│   │            │    MCP GATEWAY       │                         │       │
│   │            │    (Sidecar)         │                         │       │
│   │            └──────────┬───────────┘                         │       │
│   │                       │                                     │       │
│   │     ┌─────────────────┼─────────────────┐                   │       │
│   │     │                 │                 │                   │       │
│   │     ▼                 ▼                 ▼                   │       │
│   │ ┌────────┐      ┌────────┐      ┌────────┐                 │       │
│   │ │Pipeline│      │  Data  │      │ Model  │                 │       │
│   │ │  MCP   │      │  MCP   │      │Registry│                 │       │
│   │ │        │      │        │      │  MCP   │                 │       │
│   │ │Kubeflow│      │S3/Ceph │      │        │                 │       │
│   │ └────────┘      └────────┘      └────────┘                 │       │
│   │                                                              │       │
│   └─────────────────────────────────────────────────────────────┘       │
│                                                                          │
│   KEY PATTERNS:                                                         │
│   • MCP runs as sidecar container in pod                                │
│   • Service mesh (Istio) handles mTLS                                   │
│   • Kubernetes RBAC maps to MCP tool access                             │
│   • Pipeline MCP integrates with Kubeflow Pipelines                     │
│   • Data MCP provides secure access to S3/Ceph storage                  │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
'''

print(OPENSHIFT_PATTERN)

In [ ]:
# OpenShift AI MCP Configuration
openshift_config = {
    "apiVersion": "mcp.openshift.io/v1",
    "kind": "MCPGateway",
    "metadata": {
        "name": "ai-workbench-mcp",
        "namespace": "data-science-project"
    },
    "spec": {
        "replicas": 2,
        "selector": {
            "matchLabels": {
                "app": "mcp-gateway"
            }
        },
        "authentication": {
            "type": "serviceAccount",
            "rbacMapping": {
                "enabled": True,
                "clusterRole": "mcp-tool-access"
            }
        },
        "backends": [
            {
                "name": "pipeline-mcp",
                "service": "kubeflow-pipelines.kubeflow.svc",
                "port": 8080,
                "tools": ["create_pipeline", "run_pipeline", "get_results"]
            },
            {
                "name": "data-mcp",
                "service": "minio.storage.svc",
                "port": 9000,
                "tools": ["read_object", "write_object", "list_buckets"]
            },
            {
                "name": "model-registry-mcp",
                "service": "model-registry.mlops.svc",
                "port": 8000,
                "tools": ["register_model", "get_model", "deploy_model"]
            }
        ],
        "observability": {
            "metrics": {
                "enabled": True,
                "serviceMonitor": True
            },
            "tracing": {
                "enabled": True,
                "jaeger": "jaeger-collector.observability.svc:14268"
            }
        }
    }
}

print("OpenShift AI MCP Gateway Configuration:")
print("=" * 60)
print(json.dumps(openshift_config, indent=2))

## 6. Real-World Pattern: Apigee API Gateway

How Apigee extends its API gateway for MCP:

In [ ]:
# Apigee MCP Gateway Pattern
APIGEE_PATTERN = '''
┌─────────────────────────────────────────────────────────────────────────┐
│              APIGEE API GATEWAY - MCP EXTENSION                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────────────────────────────────────────────────────────┐       │
│   │                      APIGEE EDGE                             │       │
│   │                                                              │       │
│   │   EXISTING API MANAGEMENT          MCP EXTENSION            │       │
│   │   ════════════════════            ══════════════            │       │
│   │                                                              │       │
│   │   ┌──────────────┐              ┌──────────────┐            │       │
│   │   │   REST API   │              │  MCP PROXY   │            │       │
│   │   │   Proxies    │              │  Policies    │            │       │
│   │   └──────────────┘              └──────────────┘            │       │
│   │                                                              │       │
│   │   ┌──────────────┐              ┌──────────────┐            │       │
│   │   │   OAuth &    │              │ Tool Schema  │            │       │
│   │   │   API Keys   │◄────────────►│ Validation   │            │       │
│   │   └──────────────┘              └──────────────┘            │       │
│   │                                                              │       │
│   │   ┌──────────────┐              ┌──────────────┐            │       │
│   │   │   Rate       │              │ SSE Stream   │            │       │
│   │   │   Limiting   │◄────────────►│ Management   │            │       │
│   │   └──────────────┘              └──────────────┘            │       │
│   │                                                              │       │
│   │   ┌──────────────┐              ┌──────────────┐            │       │
│   │   │   Analytics  │              │ Tool Usage   │            │       │
│   │   │   Dashboard  │◄────────────►│ Analytics    │            │       │
│   │   └──────────────┘              └──────────────┘            │       │
│   │                                                              │       │
│   └─────────────────────────────────────────────────────────────┘       │
│                                │                                        │
│                                ▼                                        │
│   ┌────────────────────────────────────────────────────────────┐       │
│   │              BACKEND MCP SERVERS                            │       │
│   │   [CRM MCP]  [ERP MCP]  [Database MCP]  [Custom MCP]       │       │
│   └────────────────────────────────────────────────────────────┘       │
│                                                                          │
│   APIGEE ADVANTAGES FOR MCP:                                            │
│   • Reuse existing API management infrastructure                        │
│   • Unified developer portal for REST + MCP                             │
│   • Consistent security policies across all APIs                        │
│   • Built-in monetization for MCP tools                                 │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
'''

print(APIGEE_PATTERN)

## 7. Banking/Financial Services Pattern

Enterprise pattern for highly regulated environments:

In [ ]:
# Banking MCP Architecture
BANKING_PATTERN = '''
┌─────────────────────────────────────────────────────────────────────────┐
│              BANKING MCP ARCHITECTURE                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────────────────────────────────────────────────────────┐       │
│   │                    SECURITY PERIMETER                        │       │
│   │                                                              │       │
│   │   ┌──────────────┐      ┌──────────────┐                    │       │
│   │   │   WAF        │      │   DLP        │                    │       │
│   │   │   (Cloud     │      │   (Data Loss │                    │       │
│   │   │   Armor)     │      │   Prevention)│                    │       │
│   │   └──────┬───────┘      └──────┬───────┘                    │       │
│   │          └──────────┬──────────┘                            │       │
│   │                     ▼                                       │       │
│   │          ┌──────────────────┐                               │       │
│   │          │   MCP GATEWAY    │                               │       │
│   │          │   (DMZ)          │                               │       │
│   │          │                  │                               │       │
│   │          │ • OAuth 2.1      │                               │       │
│   │          │ • mTLS required  │                               │       │
│   │          │ • PCI-DSS logs   │                               │       │
│   │          └────────┬─────────┘                               │       │
│   │                   │                                         │       │
│   └───────────────────┼─────────────────────────────────────────┘       │
│                       │                                                  │
│   ┌───────────────────┼─────────────────────────────────────────┐       │
│   │   INTERNAL ZONE   │                                          │       │
│   │                   ▼                                          │       │
│   │   ┌───────────────────────────────────────────┐             │       │
│   │   │         TOOL AUTHORIZATION LAYER           │             │       │
│   │   │                                            │             │       │
│   │   │  ┌────────┐  ┌────────┐  ┌────────┐      │             │       │
│   │   │  │Approval│  │ 4-Eyes │  │ Audit  │      │             │       │
│   │   │  │Workflow│  │ Check  │  │ Trail  │      │             │       │
│   │   │  └────────┘  └────────┘  └────────┘      │             │       │
│   │   │                                            │             │       │
│   │   └───────────────────────────────────────────┘             │       │
│   │                       │                                      │       │
│   │     ┌─────────────────┼─────────────────┐                   │       │
│   │     │                 │                 │                   │       │
│   │     ▼                 ▼                 ▼                   │       │
│   │ ┌────────┐      ┌────────┐      ┌────────┐                 │       │
│   │ │Customer│      │Trading │      │Compliance│                │       │
│   │ │  MCP   │      │  MCP   │      │   MCP    │                │       │
│   │ │        │      │        │      │          │                │       │
│   │ │(read)  │      │(trade) │      │(audit)   │                │       │
│   │ └────────┘      └────────┘      └────────┘                 │       │
│   │                                                              │       │
│   └─────────────────────────────────────────────────────────────┘       │
│                                                                          │
│   BANKING REQUIREMENTS:                                                 │
│   • 4-eyes principle for sensitive operations                           │
│   • Real-time fraud detection on tool calls                             │
│   • PCI-DSS compliant logging (7 years retention)                       │
│   • Segregation of duties enforced at tool level                        │
│   • DR/BC with RPO < 1 hour, RTO < 4 hours                              │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
'''

print(BANKING_PATTERN)

In [ ]:
# Banking MCP Tools with Approval Workflow

class ApprovalStatus(Enum):
    PENDING = "pending"
    APPROVED = "approved"
    REJECTED = "rejected"
    EXPIRED = "expired"


@dataclass
class ApprovalRequest:
    """Request for approval of sensitive operation."""
    request_id: str
    tool_name: str
    arguments: Dict[str, Any]
    requester_id: str
    requester_email: str
    created_at: datetime
    expires_at: datetime
    status: ApprovalStatus = ApprovalStatus.PENDING
    approver_id: Optional[str] = None
    approved_at: Optional[datetime] = None
    rejection_reason: Optional[str] = None


class FourEyesApprovalWorkflow:
    """Implements 4-eyes principle for sensitive operations."""
    
    # Tools requiring approval
    SENSITIVE_TOOLS = {
        "execute_trade": {"threshold": 100000, "expiry_hours": 4},
        "transfer_funds": {"threshold": 50000, "expiry_hours": 2},
        "modify_account": {"threshold": 0, "expiry_hours": 24},
        "export_data": {"threshold": 0, "expiry_hours": 8},
    }
    
    def __init__(self):
        self.pending_requests: Dict[str, ApprovalRequest] = {}
    
    def requires_approval(
        self, 
        tool_name: str, 
        arguments: Dict[str, Any]
    ) -> bool:
        """Check if tool call requires approval."""
        if tool_name not in self.SENSITIVE_TOOLS:
            return False
        
        config = self.SENSITIVE_TOOLS[tool_name]
        amount = arguments.get("amount", 0)
        return amount >= config["threshold"]
    
    def create_request(
        self,
        tool_name: str,
        arguments: Dict[str, Any],
        requester_id: str,
        requester_email: str
    ) -> ApprovalRequest:
        """Create an approval request."""
        config = self.SENSITIVE_TOOLS.get(tool_name, {"expiry_hours": 24})
        
        request = ApprovalRequest(
            request_id=str(uuid.uuid4()),
            tool_name=tool_name,
            arguments=arguments,
            requester_id=requester_id,
            requester_email=requester_email,
            created_at=datetime.now(),
            expires_at=datetime.now() + timedelta(hours=config["expiry_hours"])
        )
        
        self.pending_requests[request.request_id] = request
        return request
    
    def approve(
        self,
        request_id: str,
        approver_id: str
    ) -> tuple[bool, str]:
        """Approve a request (must be different person than requester)."""
        request = self.pending_requests.get(request_id)
        if not request:
            return False, "Request not found"
        
        # 4-eyes: approver must be different from requester
        if approver_id == request.requester_id:
            return False, "Cannot approve your own request (4-eyes principle)"
        
        # Check expiry
        if datetime.now() > request.expires_at:
            request.status = ApprovalStatus.EXPIRED
            return False, "Request has expired"
        
        request.status = ApprovalStatus.APPROVED
        request.approver_id = approver_id
        request.approved_at = datetime.now()
        
        return True, "Approved"
    
    def reject(
        self,
        request_id: str,
        approver_id: str,
        reason: str
    ) -> tuple[bool, str]:
        """Reject a request."""
        request = self.pending_requests.get(request_id)
        if not request:
            return False, "Request not found"
        
        request.status = ApprovalStatus.REJECTED
        request.approver_id = approver_id
        request.rejection_reason = reason
        
        return True, "Rejected"


# Demo
workflow = FourEyesApprovalWorkflow()

print("4-Eyes Approval Workflow Demo:")
print("=" * 60)

# Trader requests a large trade
args = {"symbol": "AAPL", "quantity": 1000, "amount": 150000}
needs_approval = workflow.requires_approval("execute_trade", args)
print(f"\nTrade of $150,000 requires approval: {needs_approval}")

# Create approval request
request = workflow.create_request(
    tool_name="execute_trade",
    arguments=args,
    requester_id="trader-001",
    requester_email="trader@bank.com"
)
print(f"Created approval request: {request.request_id[:8]}...")
print(f"Expires at: {request.expires_at}")

# Try to self-approve (should fail)
success, msg = workflow.approve(request.request_id, "trader-001")
print(f"\nSelf-approval attempt: {msg}")

# Manager approves (should succeed)
success, msg = workflow.approve(request.request_id, "manager-001")
print(f"Manager approval: {msg}")
print(f"Request status: {request.status.value}")

## 8. High Availability and Disaster Recovery

### HA Architecture for MCP Gateway

In [ ]:
# High Availability Architecture
HA_ARCHITECTURE = '''
┌─────────────────────────────────────────────────────────────────────────┐
│              HIGH AVAILABILITY MCP ARCHITECTURE                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   REGION A (Primary)                    REGION B (DR)                   │
│   ══════════════════                    ═══════════════                 │
│                                                                          │
│   ┌──────────────────┐                 ┌──────────────────┐            │
│   │   Global Load    │◄───────────────►│   Global Load    │            │
│   │   Balancer       │                 │   Balancer       │            │
│   └────────┬─────────┘                 └────────┬─────────┘            │
│            │                                    │                       │
│   ┌────────┴────────┐                  ┌────────┴────────┐             │
│   │                 │                  │                 │             │
│   ▼                 ▼                  ▼                 ▼             │
│ ┌─────┐         ┌─────┐            ┌─────┐         ┌─────┐            │
│ │GW-1 │         │GW-2 │            │GW-3 │         │GW-4 │            │
│ │     │         │     │            │     │         │     │            │
│ │(Act)│         │(Act)│            │(Stb)│         │(Stb)│            │
│ └──┬──┘         └──┬──┘            └──┬──┘         └──┬──┘            │
│    │               │                  │               │                │
│    └───────┬───────┘                  └───────┬───────┘                │
│            │                                  │                        │
│            ▼                                  ▼                        │
│   ┌──────────────────┐            ┌──────────────────┐                │
│   │   MCP Backends   │            │   MCP Backends   │                │
│   │   (Primary)      │───────────►│   (Replicas)     │                │
│   └──────────────────┘  Async     └──────────────────┘                │
│            │            Repl               │                           │
│            ▼                               ▼                           │
│   ┌──────────────────┐            ┌──────────────────┐                │
│   │   Database       │───────────►│   Database       │                │
│   │   (Primary)      │  Sync Repl │   (Replica)      │                │
│   └──────────────────┘            └──────────────────┘                │
│                                                                          │
│   FAILOVER PROCEDURE:                                                   │
│   1. Health check detects Region A failure                              │
│   2. Global LB routes traffic to Region B                               │
│   3. Standby gateways become active                                     │
│   4. Database replica promoted to primary                               │
│   5. RTO: < 5 minutes, RPO: < 1 minute                                  │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
'''

print(HA_ARCHITECTURE)

## 9. Key Takeaways

### Enterprise MCP Checklist

| Category | Requirement | Implementation |
|----------|-------------|----------------|
| **Security** | Zero-trust | mTLS, OAuth 2.1, input validation |
| **Compliance** | Audit trail | Immutable logs, PII masking |
| **Multi-tenant** | Isolation | Dedicated backends or namespaces |
| **Availability** | 99.9%+ | Multi-region, auto-failover |
| **Scalability** | Horizontal | Stateless gateways, connection pooling |
| **Observability** | Full stack | Metrics, traces, structured logs |

### Pattern Selection Guide

| Requirement | Recommended Pattern |
|-------------|---------------------|
| Multi-department | Shared gateway, isolated backends |
| Strict isolation | Tenant-specific gateways |
| Global presence | Federated gateways |
| Regulated industry | Banking pattern with approvals |
| Kubernetes native | OpenShift AI pattern |
| Existing API mgmt | Apigee extension pattern |

### Next Notebook

In **Notebook 11: Observability & Debugging**, we'll explore:
- OpenTelemetry integration
- Distributed tracing for MCP
- Metrics and dashboards
- Debugging techniques

## 10. Exercises

### Exercise 1: Design Multi-Tenant Config

Create tenant configurations for:
- Free tier (limited tools, low rate limits)
- Pro tier (most tools, moderate limits)
- Enterprise tier (all tools, high limits, custom backends)

In [ ]:
# Exercise 1: Design your tenant tiers

tier_configs = {
    "free": {
        # Your configuration here
    },
    "pro": {
        # Your configuration here
    },
    "enterprise": {
        # Your configuration here
    }
}

print("Your Tenant Tier Configurations:")
print(json.dumps(tier_configs, indent=2))

### Exercise 2: Implement Compliance Report

Create a compliance report generator that includes:
- PII access summary
- Failed authentication attempts
- High-risk operations
- User activity breakdown

In [ ]:
# Exercise 2: Implement compliance report

def generate_detailed_compliance_report(
    events: List[AuditEvent],
    tenant_id: str,
    period_days: int
) -> Dict[str, Any]:
    """Generate a detailed compliance report.
    
    Include:
    - PII access summary
    - Failed authentication attempts
    - High-risk operations
    - User activity breakdown
    """
    # Your implementation here
    return {}

# Test with sample events
# report = generate_detailed_compliance_report(audit.events, "acme-corp", 30)
# print(json.dumps(report, indent=2))

### Exercise 3: Design DR Procedure

Document a disaster recovery procedure including:
- Detection criteria
- Failover steps
- Verification checklist
- Rollback procedure

In [ ]:
# Exercise 3: Document DR procedure

dr_procedure = {
    "detection": {
        "criteria": [
            # When to trigger DR
        ],
        "automated_checks": [
            # Health checks that trigger alerts
        ]
    },
    "failover_steps": [
        # Ordered steps for failover
    ],
    "verification": [
        # Checklist to verify DR succeeded
    ],
    "rollback": [
        # Steps to return to primary
    ],
    "contacts": {
        # On-call contacts
    }
}

print("Your DR Procedure:")
print(json.dumps(dr_procedure, indent=2))

---

## References

- [SOC 2 Compliance Requirements](https://www.aicpa.org/soc2)
- [PCI-DSS Standards](https://www.pcisecuritystandards.org/)
- [Red Hat OpenShift AI Documentation](https://docs.redhat.com/en/documentation/red_hat_openshift_ai_self-managed/)
- [Google Apigee Documentation](https://cloud.google.com/apigee/docs)
- [Zero Trust Architecture - NIST](https://www.nist.gov/publications/zero-trust-architecture)

---

**Next Notebook:** [11_Observability_and_Debugging.ipynb](./11_Observability_and_Debugging.ipynb)